In [263]:
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None
import time

In [264]:
def check_dupl_inn(value):
    """
    Функция для добавления нуля если количество цифр равно 9
    """
    if len(value) == 9:
        return f'0{value}'
    else:
        return value
    

In [265]:
def check_bur_inn(value:str):
    """
    Функция для проверки начинается ли инн с 03
    """
    if len(value) == 10 and value.startswith('03'):
        return f'бурятский ИНН'
    else:
        return 'неопределенный ИНН'

In [266]:
all_df = pd.read_excel('data/data.xlsx',dtype=str)

In [267]:
region = 'Республика Бурятия'
sferum_student_threshold = 33 # порог для школьников в сферуме
sferum_teacher_threshold = 51 # порог для учителей в сферуме
myschool_student_threshold = 33 # порог для школьников в моей школе
myschool_teacher_threshold = 51 # порог для учителей в моей школе


In [268]:
all_df.head(1)

,id,Субъект РФ,Наименование АТО,Краткое наименование организации,Полное наименование организации,Адрес организации,ИНН,Тип поселения,Статус организации\n(юр.лицо/филиал),Статус\nфункционирования,...,"Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа","Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа","Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа","Обучающиеся, использовавшие за последние 12 месяцев Сферум\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев Сферум\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев Сферум + МШ \nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев Сферум + МШ \nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев Сферму + МШ и Педагоги Сферум + МШ\nБолее 50% - 1\nМенее 50% - 0","Обучающиеся, использовавшие за последние 12 месяцев МШ\nБолее 50% - 1\nМенее 50% - 0","Педагоги, использовавшие за последние 12 месяцев МШ\nБолее 50% - 1\nМенее 50% - 0"
0,16675,Алтайский край,Алейский,"МБОУ ""Большепанюшевская СОШ""","МБОУ ""Большепанюшевская СОШ"" Алейского района,...","658101 Алтайский край, Алейский район, село Бо...",2231003612,сельская местность,юридическое лицо,действующая,...,0.9130434782608695,0.03333333333333333,1.0869565217391304,0,1,1,0,0,0,1


In [269]:
all_df.columns

Index(['id', 'Субъект РФ', 'Наименование АТО',
       'Краткое наименование организации', 'Полное наименование организации',
       'Адрес организации', 'ИНН', 'Тип поселения',
       'Статус организации\n(юр.лицо/филиал)', 'Статус\nфункционирования',
       'Государственная/негосударственная организация (ГОУ/НОУ)',
       'Численность обучающихся\nвсего, чел. (Раздел 1.3. стр.01 гр.3)',
       'Численность педработников - всего, чел. (Раздел 3.1 стр.06 гр.3)',
       'Обучающиеся Сферум (чел)', 'Обучающиеся МШ (чел)',
       'Обучающиеся Cферум или МШ (чел)',
       'Обучающиеся, использовавшие за последние 12 месяцев Сферум',
       ' Педагоги, использовавшие за последние 12 месяцев Сферум',
       'Обучающиеся, использовавшие за последние 12 месяцев МШ',
       ' Педагоги, использовавшие за последние 12 месяцев МШ',
       'Педагоги, использовавшие за последние 12 месяцев Сферум или МШ (чел)',
       'Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа',
   

In [270]:
all_df.shape

(38903, 32)

In [271]:
#Фильруем датафрейм

df = all_df[all_df['Субъект РФ'] == region]

In [272]:
df.shape

(464, 32)

In [273]:
df['ИНН'] = df['ИНН'].fillna('пустой') # заполняем наны 
df['ИНН'] = df['ИНН'].astype(str) # заполняем наны 

In [274]:
df['ИНН']

20549     319002490
20550     323055021
20551     302101485
20552     314004542
20553    0307008401
            ...    
21008    0323091735
21009    0323092048
21010    0323092810
21011    0323098554
21012    0323092584
Name: ИНН, Length: 464, dtype: object

In [275]:
# Обрабатываем ИНН, должно быть 10 цифр
df.insert(7,'ИНН количество цифр',df['ИНН'].apply(lambda x: len(x)))

In [276]:
# Проверям начинается ли ИНН с 03
df.insert(8,'Бурятский ИНН',df['ИНН'].apply(check_bur_inn))

In [277]:
# Добавляем нули для поиска дубликатов
df.insert(9,'Для проверки дубликатов',df['ИНН'].apply(check_dupl_inn))

In [278]:
# Получаем дубликаты ИНН
dupl_df = df[df['Для проверки дубликатов'].duplicated(keep=False)]
dupl_df.insert(0, '№ строки дубликата ', list(map(lambda x: x + 2, list(dupl_df.index))))

In [279]:
# нули
lst_check_zero = ['Обучающиеся, использовавшие за последние 12 месяцев Сферум',
                 ' Педагоги, использовавшие за последние 12 месяцев Сферум',
                 'Обучающиеся, использовавшие за последние 12 месяцев МШ',
                 ' Педагоги, использовавшие за последние 12 месяцев МШ'] # колонки надо проверить на нули

zero_df = df[df[lst_check_zero].isin(['0']).all(axis=1)]



In [280]:
# проводим разбиение по пределам
lst_category_cols = ['Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа',
                    'Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа',
                    'Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа',
                    'Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа']

df[lst_category_cols] = df[lst_category_cols].astype(float) 
df[lst_category_cols] = df[lst_category_cols].applymap(lambda x:x*100) # приводим к удобному формату

In [281]:
df[lst_category_cols]

,"Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа","Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа","Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа","Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа"
20549,66.442953,96.428571,0.000000,10.714286
20550,35.465116,48.148148,1.162791,14.814815
20551,5.725191,63.636364,11.450382,27.272727
20552,0.000000,80.000000,0.000000,2.857143
20553,0.000000,92.592593,0.000000,7.407407
...,...,...,...,...
21008,0.000000,11.111111,0.000000,22.222222
21009,54.166667,146.153846,0.320513,38.461538
21010,11.549296,110.000000,0.000000,55.000000
21011,29.310345,81.250000,4.827586,81.250000


In [282]:
df.insert(25,'Порог_Сферум_ученики',pd.cut(df['Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа'],
                                          bins=[-1,0,4,33,49.9,60,75,100,1000],
                                          labels=['0','0-4','4-33','33-49.9','50-60','60-75','75-100','Больше 100%']))

In [283]:
df.insert(27,'Порог_Сферум_педагоги',pd.cut(df['Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа'],
                                          bins=[-1,0,4,33,49.9,60,75,100,1000],
                                          labels=['0','0-4','4-33','33-49.9','50-60','60-75','75-100','Больше 100%']))

In [284]:
df.insert(29,'Порог_МШ_ученики',pd.cut(df['Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа'],
                                          bins=[-1,0,4,33,49.9,60,75,100,1000],
                                          labels=['0','0-4','4-33','33-49.9','50-60','60-75','75-100','Больше 100%']))

In [285]:
df.insert(31,'Порог_МШ_педагоги',pd.cut(df['Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа'],
                                          bins=[-1,0,4,33,49.9,60,75,100,1000],
                                          labels=['0','0-4','4-33','33-49.9','50-60','60-75','75-100','Больше 100%']))

In [287]:
# делаем сокращенную версию для удобства
short_df = df[['Наименование АТО','Краткое наименование организации','ИНН',
              'Обучающиеся, использовавшие за последние 12 месяцев Сферум % от общего числа','Порог_Сферум_ученики',
              'Педагоги, использовавшие за последние 12 месяцев Сферум % от общего числа','Порог_Сферум_педагоги',
              'Обучающиеся, использовавшие за последние 12 месяцев МШ % от общего числа','Порог_МШ_ученики',
              'Педагоги, использовавшие за последние 12 месяцев МШ % от общего числа','Порог_МШ_педагоги']]

In [288]:
# получаем время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
with pd.ExcelWriter(f'data/Проверка {current_time}.xlsx') as writer:
    df.to_excel(writer,sheet_name='Проверка ИНН',index=False)
    dupl_df.to_excel(writer,sheet_name='Дубликаты ИНН',index=False)
    zero_df.to_excel(writer,sheet_name='Нули',index=False)
    short_df.to_excel(writer,sheet_name='Пороги',index=False)

In [ ]:
'Lindy Booth'